In [27]:
!pip3 install basc-py4chan

  Preparing metadata (setup.py) ... done
  Created wheel for basc-py4chan: filename=BASC_py4chan-0.6.6-py3-none-any.whl size=14117 sha256=ab43afd0cb35c999348c7099a0d37f50d05d4e88476e35cf8d682ba5ba1207d5
  Stored in directory: /Users/dhanushkikkisetti/Library/Caches/pip/wheels/52/07/aa/28262bb2bfe6f94aef29cdc5badb583059c51ac9b116306ac1
Successfully built basc-py4chan


In [2]:
import requests
import basc_py4chan
import pandas as pd
import numpy as np
import regex as re
import datetime

# Code for webscarping


In [264]:
data=requests.get("https://boards.4chan.org/b/threads.json")



In [3]:
## Get list of all boards on 4Chan
boards_list=basc_py4chan.get_all_boards()

In [26]:
boards_list

[<Board /3/>,
 <Board /a/>,
 <Board /aco/>,
 <Board /adv/>,
 <Board /an/>,
 <Board /b/>,
 <Board /bant/>,
 <Board /biz/>,
 <Board /c/>,
 <Board /cgl/>,
 <Board /ck/>,
 <Board /cm/>,
 <Board /co/>,
 <Board /d/>,
 <Board /diy/>,
 <Board /e/>,
 <Board /f/>,
 <Board /fa/>,
 <Board /fit/>,
 <Board /g/>,
 <Board /gd/>,
 <Board /gif/>,
 <Board /h/>,
 <Board /hc/>,
 <Board /his/>,
 <Board /hm/>,
 <Board /hr/>,
 <Board /i/>,
 <Board /ic/>,
 <Board /int/>,
 <Board /jp/>,
 <Board /k/>,
 <Board /lgbt/>,
 <Board /lit/>,
 <Board /m/>,
 <Board /mlp/>,
 <Board /mu/>,
 <Board /n/>,
 <Board /news/>,
 <Board /o/>,
 <Board /out/>,
 <Board /p/>,
 <Board /po/>,
 <Board /pol/>,
 <Board /pw/>,
 <Board /qa/>,
 <Board /qst/>,
 <Board /r/>,
 <Board /r9k/>,
 <Board /s/>,
 <Board /s4s/>,
 <Board /sci/>,
 <Board /soc/>,
 <Board /sp/>,
 <Board /t/>,
 <Board /tg/>,
 <Board /toy/>,
 <Board /trash/>,
 <Board /trv/>,
 <Board /tv/>,
 <Board /u/>,
 <Board /v/>,
 <Board /vg/>,
 <Board /vip/>,
 <Board /vm/>,
 <Board /vmg/>,

In [29]:
# get the list of all the threads in political board
board = basc_py4chan.Board('pol')
thread_obs = board.get_all_threads()
#str_thread_ids = [str(id) for id in thread_ids]  # need to do this so str.join below works
#print('There are', len(all_ids), 'active threads on /tg/:', ', '.join(str_thread_ids))


In [30]:
#Get the length of all threads in the
board.title

'Politically Incorrect'

In [219]:
#initially work on one thread and then latter convert this for all the threads. 
new_thread=board.get_thread(thread_id=474990408, update_if_cached=True, raise_404=True)
#Get all posts and replies for that thread
thread_allposts=new_thread.all_posts

In [279]:
#checking all the functions the Post class has
samplepost=thread_allposts[0]
print("Post_id ",samplepost.post_id)
print("Poster_id ",samplepost.poster_id)
print("Poster name  ",samplepost.name)
print("email ",samplepost.email)
print("subject ",samplepost.subject)
print("comment ",samplepost.comment)
print("text_document ",samplepost.text_comment)
print("spoiler ",samplepost.spoiler)
print("timestamo ",samplepost.datetime)

Post_id  921768721
Poster_id  None
Poster name   Anonymous
email  None
subject  None
comment  FUCK YOUR MOTHER
text_document  FUCK YOUR MOTHER
spoiler  False
timestamo  2024-07-23 14:19:17


In [4]:
def term_in_text(text,term):
    pattern=re.compile(term)
    if pattern.search(text.lower()):
        return True
    else:
        return False

In [348]:
# creating a dataframe and storing necessary information related to post to it. 
dataset_4chan=pd.DataFrame(columns=['Post_id','User_name','Text_document','Date','Board_title'])
#Iterate through all the threads and all the posts in the thread
for thread in thread_obs:
    thread_allposts=thread.all_posts
    #Write a for loop to read each post and append its values to dataframe
    for post_ob in thread_allposts:
        values=[post_ob.post_id,post_ob.name,post_ob.text_comment,post_ob.datetime]
        if term_in_text(post_ob.text_comment,'soros'):
            dataset_4chan.loc[len(dataset_4chan)]=values
        else:
            continue

In [5]:
def extract_posts(term="NA",board='all'):
    # creating a dataframe and storing necessary information related to post to it. 
    iter=0
    dataset_4chan=pd.DataFrame(columns=['Post_id','User_name','Text_document','Date','Board_title'])
    if board=='all':
        for board_ob in boards_list:
            iter+=1
            print(iter)
            thread_obs = board_ob.get_all_threads(expand=True)
            #Write a for loop to read each post and append its values to dataframe
            if term=='NA':
                for thread in thread_obs:
                    thread_allposts=thread.all_posts
                    for post_ob in thread_allposts:
                        values=[post_ob.post_id,post_ob.name,post_ob.text_comment,post_ob.datetime,board_ob.title]
                        dataset_4chan.loc[len(dataset_4chan)]=values
            else:
                for thread in thread_obs:
                    thread_allposts=thread.all_posts
                    for post_ob in thread_allposts:
                        values=[post_ob.post_id,post_ob.name,post_ob.text_comment,post_ob.datetime,board_ob.title]
                        if term_in_text(post_ob.text_comment,term):
                            dataset_4chan.loc[len(dataset_4chan)]=values
                        else:
                            continue
   
    else:
        board = basc_py4chan.Board(board)
        thread_obs = board.get_all_threads(expand=True)

        #Iterate through all the threads and all the posts in the thread
        
        if term=='NA':
            #Write a for loop to read each post and append its values to dataframe
            for thread in thread_obs:
                thread_allposts=thread.all_posts
                for post_ob in thread_allposts:
                    values=[post_ob.post_id,post_ob.name,post_ob.text_comment,post_ob.datetime,board.title]
                    dataset_4chan.loc[len(dataset_4chan)]=values
        else:
            for thread in thread_obs:
                thread_allposts=thread.all_posts
                for post_ob in thread_allposts:
                    values=[post_ob.post_id,post_ob.name,post_ob.text_comment,post_ob.datetime,board.title]
                    if term_in_text(post_ob.text_comment,term):
                        dataset_4chan.loc[len(dataset_4chan)]=values
                    else:
                        continue
        return dataset_4chan

In [6]:
extracted_dataset=extract_posts(term='NA',board='pol')

In [7]:
extracted_dataset.shape

(11822, 5)

In [8]:
#Converting timestamp date into date
extracted_dataset['Date']=extracted_dataset['Date'].apply(lambda x:x.date())

In [78]:
extracted_dataset.head()

,Post_id,User_name,Text_document,Date,Board_title
0,478270888,Anonymous,"When it comes to wanting kids, it is due to bo...",2024-08-14,Politically Incorrect
1,478271017,Anonymous,I'd love to have at least four kids. But I wou...,2024-08-14,Politically Incorrect
2,478271162,Anonymous,>>478270888\n\nNo....modern life requires 2 wo...,2024-08-14,Politically Incorrect
3,478271189,Anonymous,>>478270888\nIt's the damn pfas in our balls. ...,2024-08-14,Politically Incorrect
4,478271305,Anonymous,>>478270888\nThe real reason most people are m...,2024-08-14,Politically Incorrect


In [9]:
print(extracted_dataset.isna().sum())
extracted_dataset=extracted_dataset.dropna()

Post_id          0
User_name        0
Text_document    0
Date             0
Board_title      0
dtype: int64


In [127]:
thread = board.get_thread(thread_ids[80])
post=thread.all_posts[1]

In [130]:
post.text_comment
post.datetime

datetime.datetime(2024, 7, 21, 16, 28, 23)

In [32]:
topic

<Post /tg/93389618#93389618, has_file: True>

In [20]:
topic=thread.topic

In [23]:
topic.comment

'What&#039;s your setting&#039;s justification for why the PC&#039;s are so much stronger than your typical NPC grunts? Some fantasy settings such as Exalted and Fire Emblem have things like the exaltations and crests designed to set certain people up to be extra powerful and special which explains how they&#039;re able to defeat scores of lesser enemies, but not all settings do this which leads to questions of why a high level Fighter is able to defeat so many regular NPC soldiers.'

In [24]:
randomdata.json()['boards']

[{'board': '3',
  'title': '3DCG',
  'ws_board': 1,
  'per_page': 15,
  'pages': 10,
  'max_filesize': 4194304,
  'max_webm_filesize': 4194304,
  'max_comment_chars': 2000,
  'max_webm_duration': 120,
  'bump_limit': 310,
  'image_limit': 150,
  'cooldowns': {'threads': 600, 'replies': 60, 'images': 60},
  'meta_description': "&quot;/3/ - 3DCG&quot; is 4chan's board for 3D modeling and imagery.",
  'is_archived': 1},
 {'board': 'a',
  'title': 'Anime & Manga',
  'ws_board': 1,
  'per_page': 15,
  'pages': 10,
  'max_filesize': 4194304,
  'max_webm_filesize': 4194304,
  'max_comment_chars': 2000,
  'max_webm_duration': 120,
  'bump_limit': 500,
  'image_limit': 300,
  'cooldowns': {'threads': 600, 'replies': 60, 'images': 60},
  'meta_description': "&quot;/a/ - Anime &amp; Manga&quot; is 4chan's imageboard dedicated to the discussion of Japanese animation and manga.",
  'spoilers': 1,
  'custom_spoilers': 1,
  'is_archived': 1},
 {'board': 'aco',
  'title': 'Adult Cartoons',
  'ws_board

# Code for database connectivity 

In [10]:
import pyodbc
import pymssql
import mysql.connector
from mysql.connector import Error


In [9]:
!pip3 install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.0/373.0 kB 7.0 MB/s eta 0:00:0000:01


In [18]:
def check_connection(host_name, user_name, user_password, database):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=database
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    finally:
        if connection.is_connected():
            connection.close()
            print("Connection closed sucessfully")

    return connection

connection = check_connection("localhost", "root", "Dh@nushk66",'test')

Connection to MySQL DB successful
Connection closed sucessfully


In [64]:
import mysql.connector

try:
    connection = mysql.connector.connect(host='localhost',
                                         database='test',
                                         user='root',
                                         password='Dh@nushk66')

    mySql_Create_Table_Query = """CREATE TABLE 4chan_new2 ( 
                             Post_ID int(20) NOT NULL,
                             User_Name varchar(250) NOT NULL,
                             Post TEXT NOT NULL,
                             Date Date NOT NULL,
                             Board_Title varchar(250) NOT NULL
                             ) """
    cursor = connection.cursor()
    result=cursor.execute(mySql_Create_Table_Query)
    print("4chan table created successfully")
except mysql.connector.Error as error:
    print("Failed to create table in MySQL:{}".format(error))


4chan table created successfully


In [11]:
#Create a list of rows to insert into table
values=[]
for i,row in extracted_dataset.iterrows():
    values.append((row.Post_id,row.User_name,row.Text_document,row.Date,row.Board_title))



In [12]:
#Inserting data into SQL table
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='test',
                                         user='root',
                                         password='Dh@nushk66')
    
    insert_sql_query=""" Insert into 4chan_new2 (Post_ID,User_Name,Post,Date,Board_Title) 
                        VALUES (%s,%s,%s,%s,%s)"""
    cursor=connection.cursor()
    cursor.executemany(insert_sql_query,values)
    connection.commit()
    print(cursor.rowcount," Records inserted sucessfully")


except mysql.connector.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))



11822  Records inserted sucessfully


In [13]:
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='test',
                                         user='root',
                                         password='Dh@nushk66')
    cursor=connection.cursor()
    sql_select_query="select * from 4chan_new2"
    cursor.execute(sql_select_query)
    records=cursor.fetchall()
    data=pd.DataFrame(records,columns=['Post_id','User_name','Text_document','Date','Board_title'])
    print(data.sample(10))
    
except mysql.connector.Error as error:
    print("Failed to read the error:{}".format(error))

         Post_id  User_name  \
12147  478284298  Anonymous   
2557   478201288  Anonymous   
8089   478191205  Anonymous   
17694  478272028  Anonymous   
13384  478274863  Anonymous   
32558  478375912  Anonymous   
29988  478375739  Anonymous   
4621   478200136  Anonymous   
16672  478277160  Anonymous   
15952  478276444  Anonymous   

                                           Text_document        Date  \
12147                           >>478283273\n>>478283223  2024-08-14   
2557   >>478200434\nEE is a shithole. But it used to ...  2024-08-13   
8089         >>478189649\nAkshually.. it started in 1945  2024-08-13   
17694  >>478252691\n>few months later it'll be healed...  2024-08-14   
13384  >>478267684\n>shit doesn't change\n>infact eve...  2024-08-14   
32558  >>478375701\nthank you for using my OC\nkeep o...  2024-08-15   
29988  >>478372777\n>isn't that just conspiracy theor...  2024-08-15   
4621                                >>478196755\nmogs me  2024-08-13   
16672  >>4

In [15]:
#Function to extract data directly from database instead of any API call
def Get_Data_From_Database(terms:list):

    try:
        connection = mysql.connector.connect(host='localhost',
                                             database='test',
                                             user='root',
                                             password='Dh@nushk66')
        cursor=connection.cursor()
        sql_select_query="select * from 4chan_new2"
        cursor.execute(sql_select_query)
        records=cursor.fetchall()
        data=pd.DataFrame(records,columns=['Post_id','User_name','Text_document','Date','Board_title'])
        print("Extracted data from database successfully")
    
    except mysql.connector.Error as error:
        print("Failed to read the error:{}".format(error))
        return
    finally:
        if connection.is_connected():
            connection.close()
            cursor.close()
            print("MySQL connection is closed")
    #Creating regex pattern
    if len(terms)==1:
        re_pattern=terms[0]
    else:
        re_pattern="|".join(str.lower(term) for term in terms)
    data=data[data['Text_document'].str.contains(r'{}'.format(re_pattern),case=False)]

    return data
        

In [16]:
data=Get_Data_From_Database(terms=['kike','cabal'])

Extracted data from database successfully
MySQL connection is closed


In [17]:
data

,Post_id,User_name,Text_document,Date,Board_title
226,478207133,Anonymous,>>478207000\n>I am White\n>he says as he calls...,2024-08-13,Politically Incorrect
302,478208480,Anonymous,>>478208053\nNormies agent as clueless as /pol...,2024-08-13,Politically Incorrect
314,478207872,Anonymous,>>478206323\nit means \nSHUT YOUR FUCKING MOUT...,2024-08-13,Politically Incorrect
557,478189473,Anonymous,">>478189244\nWrong, kikes like them since they...",2024-08-13,Politically Incorrect
593,478195806,Anonymous,>>478195116\nBrownoid shitskin kikes are your ...,2024-08-13,Politically Incorrect
...,...,...,...,...,...
34564,478352923,Anonymous,>>478348493\nBetter than kikes and faggots,2024-08-15,Politically Incorrect
34673,478367917,Anonymous,>>478353638\n>>478353214\n>>478352934\n>>47835...,2024-08-15,Politically Incorrect
34721,478375749,Anonymous,">>478357679\nNo, Poland, much like majority of...",2024-08-15,Politically Incorrect
34732,478374694,Anonymous,>>478374061\n>The Jews frees the Neet streamer...,2024-08-15,Politically Incorrect


In [4]:
conn=pymssql.connect(
    server='localhost',
    user='root',
    password='Dh@nushk66',
    database='test',
    as_dict=True
)

OperationalError: (20009, b'DB-Lib error message 20009, severity 9:\nUnable to connect: Adaptive Server is unavailable or does not exist (localhost)\nNet-Lib error during Connection refused (61)\nDB-Lib error message 20009, severity 9:\nUnable to connect: Adaptive Server is unavailable or does not exist (localhost)\nNet-Lib error during Connection refused (61)\n')

In [2]:
con=pyodbc.connect("Driver={ODBC Driver 18 for SQL Server};Server=localhost;Database=test;UID=root;PWD=Dh@nushk66")

OperationalError: ('HYT00', '[HYT00] [Microsoft][ODBC Driver 18 for SQL Server]Login timeout expired (0) (SQLDriverConnect)')